### Bibliotecas usadas para tratamento

In [1]:
import pandas as pd
import numpy as np
import unidecode

##### Mudando configurações de visualização de tabela

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Leitura do arquivo

In [3]:
df = pd.read_parquet('CAD_UNIC_FEV_DT_23_03_05_23.parquet')

### Filtrando por renda per capita até R$ 89,00

In [4]:
df_renda_pobreza_menor_89 = df[(df['d.vlr_renda_media_fam'] <= 89)]

### Filtrando por renda per capita acima de R$ 89,00 e abaixo de R$ 218,00

In [5]:
df_renda_pobreza_maior_89 = df[(df['d.vlr_renda_media_fam'] > 89)]

##### Selecionando colunas a ser utilizadas

In [6]:
df_analise_1 = df_renda_pobreza_menor_89[['d.cod_familiar_fam', 'd.cod_material_piso_fam', 'd.cod_material_domic_fam',
                                          'd.cod_banheiro_domic_fam', 'p.cod_sexo_pessoa', 'p.cod_deficiencia_memb', 
                                          'p.cod_raca_cor_pessoa', 'p.fx_idade', 'd.qtd_comodos_domic_fam', 
                                          'd.num_cep_logradouro_fam', 'd.qtd_comodos_dormitorio_fam']]

df_analise_2 = df_renda_pobreza_maior_89[['d.cod_familiar_fam', 'd.cod_material_piso_fam', 'd.cod_material_domic_fam',
                                          'd.cod_banheiro_domic_fam', 'p.cod_sexo_pessoa', 'p.cod_deficiencia_memb', 
                                          'p.cod_raca_cor_pessoa', 'p.fx_idade', 'd.qtd_comodos_domic_fam', 
                                          'd.num_cep_logradouro_fam', 'd.qtd_comodos_dormitorio_fam']]

### Criando novo DataFrame que trás as informações necessárias em True ou False e o código familiar dos domicílios que com renda per capita até R$89,00

In [7]:
df_analise_final_menor_89 = df_analise_1.groupby('d.cod_familiar_fam')\
    .agg({'d.cod_material_domic_fam': [lambda x: 2 in x.values, lambda x: 1 in x.values], # 2 sem e 1 com
         'd.cod_material_piso_fam': [lambda x: 2 in x.values, lambda x: 1 in x.values, lambda x: 5 in x.values],# 2-Cimento|1-Terra|3-ceramica e tals
         'd.cod_banheiro_domic_fam': [lambda x: 1 in x.values, lambda x: 2 in x.values], #1-Sim|2-Não
         'p.cod_raca_cor_pessoa': [lambda x: 2 in x.values, lambda x: 4 in x.values],
         'p.cod_sexo_pessoa': lambda x: (2 == x.values).all(),
         'p.fx_idade': lambda x: 0 in x.values,
         'p.cod_deficiencia_memb': lambda x: 1 in x.values,
         'd.qtd_comodos_domic_fam': pd.Series.mode,
         'd.qtd_comodos_dormitorio_fam': pd.Series.mode,
         'd.num_cep_logradouro_fam': pd.Series.mode})\
    .rename(columns = {'d.cod_material_domic_fam': 'Alvenaria'}).reset_index()

### Ajustando, renomeando colunas e trocando True por 1 e False por 0

In [8]:
df_analise_final_menor_89.columns = ['_'.join(col) for col in df_analise_final_menor_89.columns]
df_analise_final_menor_89.columns=['d.cod_familiar_fam', 'sem_revestimento', 'com_revestimento',
                      'piso_cimento', 'piso_terra', 'piso_ceramica_afins', 'banheiro', 'sem_banheiro',
                      'pretos', 'pardos', 'familia_de_mulheres' ,'primeira_infancia', 'pcd',
                      'qnt_comodos', 'qnt_dormitorios', 'cep']
df_analise_final_menor_89 = df_analise_final_menor_89.replace(True, 1)
df_analise_final_menor_89 = df_analise_final_menor_89.replace(False, 0)

### Criando novo DataFrame que trás as informações necessárias em True ou False e o código familiar dos domicílios com renda per capita acima de R$89,00 até R$218,00

In [9]:
df_analise_final_maior_89 = df_analise_2.groupby('d.cod_familiar_fam')\
    .agg({'d.cod_material_domic_fam': [lambda x: 2 in x.values, lambda x: 1 in x.values], # 2 sem e 1 com
         'd.cod_material_piso_fam': [lambda x: 2 in x.values, lambda x: 1 in x.values, lambda x: 5 in x.values],# 2-Cimento|1-Terra|3-ceramica e tals
         'd.cod_banheiro_domic_fam': [lambda x: 1 in x.values, lambda x: 2 in x.values], #1-Sim|2-Não
         'p.cod_raca_cor_pessoa': [lambda x: 2 in x.values, lambda x: 4 in x.values],
         'p.cod_sexo_pessoa': lambda x: (2 == x.values).all(),
         'p.fx_idade': lambda x: 0 in x.values,
         'p.cod_deficiencia_memb': lambda x: 1 in x.values,
         'd.qtd_comodos_domic_fam': pd.Series.mode,
         'd.qtd_comodos_dormitorio_fam': pd.Series.mode,
         'd.num_cep_logradouro_fam': pd.Series.mode})\
    .rename(columns = {'d.cod_material_domic_fam': 'Alvenaria'}).reset_index()

### Ajustando, renomeando colunas e trocando True por 1 e False por 0

In [10]:
df_analise_final_maior_89.columns = ['_'.join(col) for col in df_analise_final_maior_89.columns]
df_analise_final_maior_89.columns=['d.cod_familiar_fam', 'sem_revestimento', 'com_revestimento',
                      'piso_cimento', 'piso_terra', 'piso_ceramica_afins', 'banheiro', 'sem_banheiro',
                      'pretos', 'pardos', 'familia_de_mulheres' ,'primeira_infancia', 'pcd',
                      'qnt_comodos', 'qnt_dormitorios', 'cep']
df_analise_final_maior_89 = df_analise_final_maior_89.replace(True, 1)
df_analise_final_maior_89 = df_analise_final_maior_89.replace(False, 0)

### Ajustando coluna 'cep' para 'Int64' e remoção de linha Null

In [11]:
df_analise_final_maior_89['cep'] = df_analise_final_maior_89['cep'].astype(str)
df_analise_final_maior_89['cep'] = df_analise_final_maior_89['cep'].str.split('.').str[0]

df_analise_final_menor_89['cep'] = df_analise_final_menor_89['cep'].astype(str)
df_analise_final_menor_89 = df_analise_final_menor_89.drop(71005)
df_analise_final_menor_89['cep'] = df_analise_final_menor_89['cep'].str.split('.').str[0]

In [12]:
df_analise_final_menor_89['cep'] = df_analise_final_menor_89['cep'].astype('Int64')
df_analise_final_maior_89['cep'] = df_analise_final_maior_89['cep'].astype('Int64')

### Lendo arquivo onde tem os CEPs que estão no ProMorar

In [13]:
df_cis_remove = pd.read_csv('CIS_REMOCAO_PRO-MORAR_HABITACIONAIS.csv')

### Ajustes e tratamento da coluna 'Cep' do ProMorar

In [14]:
df_cis_remove = df_cis_remove[['Cep']]
df_cis_remove['Cep'] = df_cis_remove['Cep'].str.replace('-', '')
df_cis_remove['Cep'] = df_cis_remove['Cep'].str.replace('.', '')
df_cis_remove['Cep'] = df_cis_remove['Cep'].astype('Int64')

C:\Users\alexandre.goncalves\AppData\Local\Temp\ipykernel_23808\1225615319.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_cis_remove['Cep'] = df_cis_remove['Cep'].str.replace('.', '')


### Removendo CEPs do ProMorar do DataFrame com informações do CadÚnico

In [15]:
df_analise_final_menor_89 = df_analise_final_menor_89[~df_analise_final_menor_89['cep'].isin(df_cis_remove['Cep'])]
df_analise_final_maior_89 = df_analise_final_maior_89[~df_analise_final_maior_89['cep'].isin(df_cis_remove['Cep'])]

### Filtro 1 - Trazendo todas as famílias que não possuem banheiro com renda acima de R$89,00 até R$218,00

In [16]:
df_pt1 = df_analise_final_maior_89[df_analise_final_maior_89['sem_banheiro']==1]

##### Criando colunas com código da obra e informações de renda

In [17]:
df_pt1['TP-04'] = 0
df_pt1['TP-05'] = 0
df_pt1['TP-06'] = 0
df_pt1['TP-07'] = 0
df_pt1['TC-09'] = 1
df_pt1['TC-10'] = 0
df_pt1['TC-12'] = 0
df_pt1['mais_89'] = 1
df_pt1['menos_89'] = 0

C:\Users\alexandre.goncalves\AppData\Local\Temp\ipykernel_23808\4266482083.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pt1['TP-04'] = 0
C:\Users\alexandre.goncalves\AppData\Local\Temp\ipykernel_23808\4266482083.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pt1['TP-05'] = 0
C:\Users\alexandre.goncalves\AppData\Local\Temp\ipykernel_23808\4266482083.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

##### Criando colunas que contém custo das obras com seus respectivos parâmetros

In [18]:
df_pt1['vlr_TP-04'] = 2161.15 * df_pt1['TP-04']
df_pt1['vlr_TP-05'] = 716.05 * df_pt1['TP-05'] * df_pt1['qnt_dormitorios']
df_pt1['vlr_TP-06'] = 854.43 * df_pt1['TP-06'] * df_pt1['qnt_comodos']
df_pt1['vlr_TP-07'] = 1995.82 * df_pt1['TP-07'] * df_pt1['qnt_comodos']
df_pt1['vlr_TC-09'] = 12094.26 * df_pt1['TC-09']
df_pt1['vlr_TC-10'] = 4334.79 * df_pt1['TC-10']
df_pt1['vlr_TC-12'] = 5579.95 * df_pt1['TC-12']

C:\Users\alexandre.goncalves\AppData\Local\Temp\ipykernel_23808\1612351869.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pt1['vlr_TP-04'] = 2161.15 * df_pt1['TP-04']
C:\Users\alexandre.goncalves\AppData\Local\Temp\ipykernel_23808\1612351869.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pt1['vlr_TP-05'] = 716.05 * df_pt1['TP-05'] * df_pt1['qnt_dormitorios']
C:\Users\alexandre.goncalves\AppData\Local\Temp\ipykernel_23808\1612351869.py:3: SettingWithCopyWarning: 
A value is trying to be set o

##### Criando coluna com o custo total da obra de cada domicílio

In [19]:
df_pt1['vlr_total'] = df_pt1['vlr_TP-04'] + \
                        df_pt1['vlr_TP-05'] + \
                        df_pt1['vlr_TP-06'] + \
                        df_pt1['vlr_TP-07'] + \
                        df_pt1['vlr_TC-09'] + \
                        df_pt1['vlr_TC-10'] + \
                        df_pt1['vlr_TC-12']

C:\Users\alexandre.goncalves\AppData\Local\Temp\ipykernel_23808\3053910290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pt1['vlr_total'] = df_pt1['vlr_TP-04'] + \


### Filtro 2 - Trazendo domicílios sem banheiro e com piso em terra com renda até R$89,00

In [20]:
df_pt2 = df_analise_final_menor_89[(df_analise_final_menor_89['sem_banheiro']==1) & (df_analise_final_menor_89['piso_terra']==1)]

##### Removendo os domicílios que foram selecionados no 'Filtro 2' do DataFrame principal

In [21]:
df_analise_final_menor_89 = df_analise_final_menor_89[~df_analise_final_menor_89['d.cod_familiar_fam'].isin(df_pt2['d.cod_familiar_fam'])]

##### Criando colunas com código da obra e informações de renda
##### Criando colunas que contém custo das obras com seus respectivos parâmetros
##### Criando coluna com o custo total da obra de cada domicílio

In [22]:
df_pt2['TP-04'] = 1
df_pt2['TP-05'] = 1
df_pt2['TP-06'] = 1
df_pt2['TP-07'] = 1
df_pt2['TC-09'] = 1
df_pt2['TC-10'] = 0
df_pt2['TC-12'] = 1
df_pt2['mais_89'] = 0
df_pt2['menos_89'] = 1

df_pt2['vlr_TP-04'] = 2161.15 * df_pt2['TP-04']
df_pt2['vlr_TP-05'] = 716.05 * df_pt2['TP-05'] * df_pt2['qnt_dormitorios']
df_pt2['vlr_TP-06'] = 854.43 * df_pt2['TP-06'] * df_pt2['qnt_comodos']
df_pt2['vlr_TP-07'] = 1995.82 * df_pt2['TP-07'] * df_pt2['qnt_comodos']
df_pt2['vlr_TC-09'] = 12094.26 * df_pt2['TC-09']
df_pt2['vlr_TC-10'] = 4334.79 * df_pt2['TC-10']
df_pt2['vlr_TC-12'] = 5579.95 * df_pt2['TC-12']

df_pt2['vlr_total'] = df_pt2['vlr_TP-04'] + \
                        df_pt2['vlr_TP-05'] + \
                        df_pt2['vlr_TP-06'] + \
                        df_pt2['vlr_TP-07'] + \
                        df_pt2['vlr_TC-09'] + \
                        df_pt2['vlr_TC-10'] + \
                        df_pt2['vlr_TC-12']

### Filtro 3 - Trazendo domicílios sem banheiro e com piso em cimento com renda até R$89,00

In [24]:
df_pt3 = df_analise_final_menor_89[(df_analise_final_menor_89['piso_cimento']==1) & (df_analise_final_menor_89['sem_banheiro']==1)]

##### Removendo os domicílios que foram selecionados no 'Filtro 3' do DataFrame principal

In [25]:
df_analise_final_menor_89 = df_analise_final_menor_89[~df_analise_final_menor_89['d.cod_familiar_fam'].isin(df_pt3['d.cod_familiar_fam'])]

##### Criando colunas com código da obra e informações de renda
##### Criando colunas que contém custo das obras com seus respectivos parâmetros
##### Criando coluna com o custo total da obra de cada domicílio

In [26]:
df_pt3['TP-04'] = 1
df_pt3['TP-05'] = 1
df_pt3['TP-06'] = 1
df_pt3['TP-07'] = 1
df_pt3['TC-09'] = 1
df_pt3['TC-10'] = 0
df_pt3['TC-12'] = 1
df_pt3['mais_89'] = 0
df_pt3['menos_89'] = 1

df_pt3['vlr_TP-04'] = 2161.15 * df_pt3['TP-04']
df_pt3['vlr_TP-05'] = 716.05 * df_pt3['TP-05'] * df_pt3['qnt_dormitorios']
df_pt3['vlr_TP-06'] = 854.43 * df_pt3['TP-06'] * df_pt3['qnt_comodos']
df_pt3['vlr_TP-07'] = 1995.82 * df_pt3['TP-07'] * df_pt3['qnt_comodos']
df_pt3['vlr_TC-09'] = 12094.26 * df_pt3['TC-09']
df_pt3['vlr_TC-10'] = 4334.79 * df_pt3['TC-10']
df_pt3['vlr_TC-12'] = 5579.95 * df_pt3['TC-12']

df_pt3['vlr_total'] = df_pt3['vlr_TP-04'] + \
                        df_pt3['vlr_TP-05'] + \
                        df_pt3['vlr_TP-06'] + \
                        df_pt3['vlr_TP-07'] + \
                        df_pt3['vlr_TC-09'] + \
                        df_pt3['vlr_TC-10'] + \
                        df_pt3['vlr_TC-12']

### Filtro 4 - Trazendo domicílios sem banheiro com renda até R$89,00

In [27]:
df_pt4 = df_analise_final_menor_89[(df_analise_final_menor_89['sem_banheiro']==1)]

##### Removendo os domicílios que foram selecionados no 'Filtro 4' do DataFrame principal

In [28]:
df_analise_final_menor_89 = df_analise_final_menor_89[~df_analise_final_menor_89['d.cod_familiar_fam'].isin(df_pt4['d.cod_familiar_fam'])]

##### Criando colunas com código da obra e informações de renda
##### Criando colunas que contém custo das obras com seus respectivos parâmetros
##### Criando coluna com o custo total da obra de cada domicílio

In [29]:
df_pt4['TP-04'] = 0
df_pt4['TP-05'] = 0
df_pt4['TP-06'] = 0
df_pt4['TP-07'] = 0
df_pt4['TC-09'] = 1
df_pt4['TC-10'] = 0
df_pt4['TC-12'] = 0
df_pt4['mais_89'] = 0
df_pt4['menos_89'] = 1

df_pt4['vlr_TP-04'] = 2161.15 * df_pt4['TP-04']
df_pt4['vlr_TP-05'] = 716.05 * df_pt4['TP-05'] * df_pt4['qnt_dormitorios']
df_pt4['vlr_TP-06'] = 854.43 * df_pt4['TP-06'] * df_pt4['qnt_comodos']
df_pt4['vlr_TP-07'] = 1995.82 * df_pt4['TP-07'] * df_pt4['qnt_comodos']
df_pt4['vlr_TC-09'] = 12094.26 * df_pt4['TC-09']
df_pt4['vlr_TC-10'] = 4334.79 * df_pt4['TC-10']
df_pt4['vlr_TC-12'] = 5579.95 * df_pt4['TC-12']

df_pt4['vlr_total'] = df_pt4['vlr_TP-04'] + \
                        df_pt4['vlr_TP-05'] + \
                        df_pt4['vlr_TP-06'] + \
                        df_pt4['vlr_TP-07'] + \
                        df_pt4['vlr_TC-09'] + \
                        df_pt4['vlr_TC-10'] + \
                        df_pt4['vlr_TC-12']

### Filtro 5 - Trazendo domicílios com piso em terra com renda até R$89,00

In [30]:
df_pt5 = df_analise_final_menor_89[(df_analise_final_menor_89['piso_terra']==1)]

##### Removendo os domicílios que foram selecionados no 'Filtro 5' do DataFrame principal

In [31]:
df_analise_final_menor_89 = df_analise_final_menor_89[~df_analise_final_menor_89['d.cod_familiar_fam'].isin(df_pt5['d.cod_familiar_fam'])]

##### Criando colunas com código da obra e informações de renda
##### Criando colunas que contém custo das obras com seus respectivos parâmetros
##### Criando coluna com o custo total da obra de cada domicílio

In [32]:
df_pt5['TP-04'] = 1
df_pt5['TP-05'] = 1
df_pt5['TP-06'] = 1
df_pt5['TP-07'] = 1
df_pt5['TC-09'] = 0
df_pt5['TC-10'] = 1
df_pt5['TC-12'] = 1
df_pt5['mais_89'] = 0
df_pt5['menos_89'] = 1

df_pt5['vlr_TP-04'] = 2161.15 * df_pt5['TP-04']
df_pt5['vlr_TP-05'] = 716.05 * df_pt5['TP-05'] * df_pt5['qnt_dormitorios']
df_pt5['vlr_TP-06'] = 854.43 * df_pt5['TP-06'] * df_pt5['qnt_comodos']
df_pt5['vlr_TP-07'] = 1995.82 * df_pt5['TP-07'] * df_pt5['qnt_comodos']
df_pt5['vlr_TC-09'] = 12094.26 * df_pt5['TC-09']
df_pt5['vlr_TC-10'] = 4334.79 * df_pt5['TC-10']
df_pt5['vlr_TC-12'] = 5579.95 * df_pt5['TC-12']

df_pt5['vlr_total'] = df_pt5['vlr_TP-04'] + \
                        df_pt5['vlr_TP-05'] + \
                        df_pt5['vlr_TP-06'] + \
                        df_pt5['vlr_TP-07'] + \
                        df_pt5['vlr_TC-09'] + \
                        df_pt5['vlr_TC-10'] + \
                        df_pt5['vlr_TC-12']

Juntando filtros 1, 2, 3, 4 e 5 em um DataFrame

In [40]:
df_final = pd.concat([df_pt1, df_pt2, df_pt3, df_pt4, df_pt5])

In [41]:
df_final

,d.cod_familiar_fam,sem_revestimento,com_revestimento,piso_cimento,piso_terra,piso_ceramica_afins,banheiro,sem_banheiro,pretos,pardos,familia_de_mulheres,primeira_infancia,pcd,qnt_comodos,qnt_dormitorios,cep,TP-04,TP-05,TP-06,TP-07,TC-09,TC-10,TC-12,mais_89,menos_89,vlr_TP-04,vlr_TP-05,vlr_TP-06,vlr_TP-07,vlr_TC-09,vlr_TC-10,vlr_TC-12,vlr_total
101,00268953406,1,0,1,0,0,0,1,0,0,0,0,0,2.0,1,50721080,0,0,0,0,1,0,0,1,0,0.00,0.0,0.0,0.0,12094.26,0.00,0.00,12094.26
126,00283621877,1,0,1,0,0,0,1,0,1,1,0,1,2.0,1,51220230,0,0,0,0,1,0,0,1,0,0.00,0.0,0.0,0.0,12094.26,0.00,0.00,12094.26
185,00337495076,0,1,1,0,0,0,1,0,1,1,0,0,1,1,50870495,0,0,0,0,1,0,0,1,0,0.00,0.0,0.0,0.0,12094.26,0.00,0.00,12094.26
226,00337530246,0,0,1,0,0,0,1,0,1,1,0,0,4.0,1,51170650,0,0,0,0,1,0,0,1,0,0.00,0.0,0.0,0.0,12094.26,0.00,0.00,12094.26
249,00337545510,0,0,0,0,0,0,1,1,0,0,0,0,4.0,1,51011330,0,0,0,0,1,0,0,1,0,0.00,0.0,0.0,0.0,12094.26,0.00,0.00,12094.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77097,07805297231,0,0,0,1,0,1,0,1,0,0,0,0,2.0,1,50820500,1,1,1,1,0,1,1,0,1,2161.15,716.05,1708.86,3991.64,0.00,4334.79,5579.95,18492.44
77252,07830807787,1,0,0,1,0,1,0,0,1,0,0,0,2.0,1,51240420,1,1,1,1,0,1,1,0,1,2161.15,716.05,1708.86,3991.64,0.00,4334.79,5579.95,18492.44
77261,07832472308,0,0,0,1,0,1,0,0,1,0,0,1,2.0,1,51010100,1,1,1,1,0,1,1,0,1,2161.15,716.05,1708.86,3991.64,0.00,4334.79,5579.95,18492.44
77266,07832869100,0,1,0,1,0,1,0,1,0,0,0,0,4.0,1,52030130,1,1,1,1,0,1,1,0,1,2161.15,716.05,3417.72,7983.28,0.00,4334.79,5579.95,24192.94


##### Verificando custo das obras

In [42]:
df_final.agg({'vlr_total': ['max', 'min']})

,vlr_total
max,43353.41
min,12094.26


In [43]:
df_final[df_final['vlr_total'] == 38368.96]

,d.cod_familiar_fam,sem_revestimento,com_revestimento,piso_cimento,piso_terra,piso_ceramica_afins,banheiro,sem_banheiro,pretos,pardos,familia_de_mulheres,primeira_infancia,pcd,qnt_comodos,qnt_dormitorios,cep,TP-04,TP-05,TP-06,TP-07,TC-09,TC-10,TC-12,mais_89,menos_89,vlr_TP-04,vlr_TP-05,vlr_TP-06,vlr_TP-07,vlr_TC-09,vlr_TC-10,vlr_TC-12,vlr_total
5192,01346005923,0,0,0,1,0,0,1,0,1,1,0,0,6.0,2.0,50761130,1,1,1,1,1,0,1,0,1,2161.15,1432.1,5126.58,11974.92,12094.26,0.0,5579.95,38368.96


In [ ]:
a = df_final.agg({'vlr_total': 'sum'})
a.apply(lambda x: '%2f' % x)

#129.339.451,28

##### Número de domicílios selecionados

In [44]:
len(df_final)

7356

In [46]:
df_final.to_excel('DF_SELECAO_HABITACIONAIS_SEM_BAIRROS.xlsx', index=False)

# Trazendo Bairros e RPA

### Lendo arquivo que contém os Bairros ajustados de cada domicílios

In [47]:
df1 = pd.read_parquet('CAD_UNIC_MAI_DT_23_04_05_23.parquet')

### GroupBy para trazer o bairro de cada domicílio

In [48]:
df1_merge = df1.groupby('d.cod_familiar_fam').agg({'bairro': pd.Series.mode}).reset_index()

### Transformando colunas do código familiar em 'Int64'

In [53]:
df1_merge['d.cod_familiar_fam'] = df1_merge['d.cod_familiar_fam'].astype('Int64')
df_final['d.cod_familiar_fam'] = df_final['d.cod_familiar_fam'].astype('Int64')

### Merge (ou join) para trazer os respectivos bairros

In [54]:
newdf = df_final.merge(df1_merge, how='left', on='d.cod_familiar_fam')

### Lendo arquivos que contém o bairro e RPA

In [56]:
df_bairros = pd.read_csv('BAIRRO.csv', sep=';')

##### Renomeando coluna de 'Bairro' para 'bairro'

In [57]:
df_bairros.rename(columns={'Bairro': 'bairro'}, inplace=True)

### Tirando acentos e caractéres especiais

In [58]:
newdf['bairro'] = newdf['bairro'].apply(lambda x: unidecode.unidecode(x) if type(x) == str else np.nan)

### Merge (ou join) para trazer os respectivos RPAs

In [59]:
newdf2 = newdf.merge(df_bairros, how='inner', on='bairro')

### Salvando arquivo com bairros e RPA

In [61]:
newdf2.to_excel('DF_SELECAO_HABITACIONAIS_COM_BAIRROS.xlsx', index=False)